In [1]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np

url = 'https://upload.wikimedia.org/wikipedia/commons/f/f2/Brachyscome_iberidifolia_Toucan_Tango_2zz.jpg'

# Wikimedia access requires a user agent
headers = {
    'User-Agent': 'DataDiary/0.1 (jared@datadiary.dev)'
}
response = requests.get(url, headers=headers)
image = Image.open(BytesIO(response.content))
image = image.convert('RGB')

image_array = np.array(image)
height = image_array.shape[0]
width = image_array.shape[1]
image_array_flat = image_array.flatten().reshape((
    height * width,
    3
))

view = image_array_flat.view((
    np.void,
    image_array_flat.dtype.itemsize * image_array_flat.shape[1]
))
num_colors = len(np.unique(view))

print(image_array_flat.shape, num_colors)

(665000, 3) 141862


In [93]:
image_array_flat.dtype

dtype('uint8')

In [112]:
def KMeans(X, n_clusters, max_iter=100, tol=1e-4):
    # initialize centroids at positions of random pixels
    centroids = X[np.random.choice(X.shape[0], n_clusters, replace=False)]

    for _ in range(max_iter):
        # assign each pixel to the nearest centroid
        distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)

        # update each centroid to the mean of assigned pixels
        new_centroids = np.array([X[labels == k].mean(axis=0) for k in range(n_clusters)])

        # check for convergence
        if np.all(np.linalg.norm(new_centroids - centroids, axis=1) < tol):
            break

        centroids = new_centroids

    return centroids, labels

n_clusters = 3
centroids, labels = KMeans(image_array_flat, n_clusters)
centroids = np.round(centroids)

# replace pixels with representative centroids
quantized_image_array_flat = centroids[labels]

quantized_image_array = quantized_image_array_flat.reshape(image_array.shape)
quantized_image = Image.fromarray(quantized_image_array.astype('uint8'), 'RGB')

quantized_image.save('Brachyscome_iberidifolia_Toucan_Tango_2zz_KMEANS3.png', 'PNG')
quantized_image.show()

In [113]:
import plotly.graph_objects as go

# sample 10k pixels to appear in the visual
image_array_flat_sample = image_array_flat[np.random.choice(image_array_flat.shape[0], 10_000, replace=False)]

# plot pixels
fig = go.Figure(data=[go.Scatter3d(
    x=image_array_flat_sample[:, 0],
    y=image_array_flat_sample[:, 1],
    z=image_array_flat_sample[:, 2],
    mode='markers',
    marker=dict(
        size=1.5,
        color=[f'rgb({pixel[0]}, {pixel[1]}, {pixel[2]})' for pixel in image_array_flat_sample],
        opacity=1.0,
        line=dict(width=0)
    ),
    hovertemplate = '<b>R</b>: %{x}<br><b>G</b>: %{y}<br><b>B</b>: %{z}<extra></extra>',
    showlegend=True,
    name='pixels'
)])

# plot centroids
fig.add_trace(go.Scatter3d(
    x=centroids[:, 0],
    y=centroids[:, 1],
    z=centroids[:, 2],
    mode='markers',
    marker=dict(
        size=8,
        color=[f'rgb({centroid[0]}, {centroid[1]}, {centroid[2]})' for centroid in centroids],
        opacity=1.0,
        symbol='x',
        line=dict(width=0)
    ),
    hovertemplate = '<b>R</b>: %{x}<br><b>G</b>: %{y}<br><b>B</b>: %{z}<extra></extra>',
    showlegend=True,
    name='centroids'
))

fig.update_layout(
    uirevision='constant',
    scene=dict(
        xaxis=dict(title='R', showspikes=False, range=[0, 255]),
        yaxis=dict(title='G', showspikes=False, range=[0, 255]),
        zaxis=dict(title='B', showspikes=False, range=[0, 255])
    ),
    scene_aspectmode='manual', 
    scene_aspectratio=dict(x=1, y=1, z=1)
)

fig.show()

with open('centroids_in_color_space.json', 'w') as outfile:
    outfile.write(fig.to_json())

In [114]:
n_clusters = 10
centroids, labels = KMeans(image_array_flat, n_clusters)
centroids = np.round(centroids)

# replace pixels with representative centroids
quantized_image_array_flat = centroids[labels]

quantized_image_array = quantized_image_array_flat.reshape(image_array.shape)
quantized_image = Image.fromarray(quantized_image_array.astype('uint8'), 'RGB')

quantized_image.save('Brachyscome_iberidifolia_Toucan_Tango_2zz_KMEANS10.png', 'PNG')
quantized_image.show()